In [1]:
# CS585 HW2
# Group member: Deyan Hao, Yirong Zhang
import cv2
import sys
import numpy as np

In [2]:
def boundingbox(src):

    # Find contours
	# Documentation for finding contours: http://docs.opencv.org/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=findcontours#findcontours
    contours, hierarchy = cv2.findContours(src, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour_output = cv2.cvtColor(np.zeros(np.shape(src), dtype='uint8'), cv2.COLOR_GRAY2BGR)

    # Fine largest contour
    if (len(contours) > 0):
        max_id = max(enumerate(contours), key=lambda x : cv2.contourArea(x[1]))[0]
        max_size = cv2.contourArea(contours[max_id])
        boundrec = cv2.boundingRect(contours[max_id])
        x,y,w,h = boundrec

        # Draw contours(ONLY the biggest one is drawn)
	    # Documentation for drawing contours: http://docs.opencv.org/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=drawcontours#drawcontours
        cv2.drawContours(contour_output, contours, max_id, (255, 255, 255), 2, 8)

        # Documentation for drawing rectangle: http://docs.opencv.org/modules/core/doc/drawing_functions.html
        cv2.rectangle(contour_output, boundrec, (0, 0, 255), 1, 8, 0)

        # Get the moments of the largest contour to compute the centroid
        M = cv2.moments(contours[max_id])
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])      
        centroid = "centroid: "+str(cx)+" ,"+str(cy)
        
        cv2.putText(contour_output,centroid,(100,100),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)

        
    return contour_output,cx,cy
    
def my_template_match(src):
    
    #read the templates and convert them to grayscale image
    five = cv2.imread('five.png')
    five = cv2.cvtColor(five, cv2.COLOR_BGR2GRAY)
    six = cv2.imread('six.png')
    six = cv2.cvtColor(six, cv2.COLOR_BGR2GRAY)
    two = cv2.imread('two.png')
    two = cv2.cvtColor(two, cv2.COLOR_BGR2GRAY)
    rock = cv2.imread('rock.png')
    rock = cv2.cvtColor(rock, cv2.COLOR_BGR2GRAY)
    
    templates = [five,six,two,rock]
    name = ["five","six","two","rock"]
    value = []

    # create src image pyramids with smaller sizes
    down1 = cv2.pyrDown(src)
#     cv2.imwrite('down1.png',down1)
#     print(down1.shape)
    down2 = cv2.pyrDown(down1)
#     cv2.imwrite('down2.png',down2)
#     print(down2.shape)
    down3 = cv2.pyrDown(down2)
#     cv2.imwrite('down3.png',down3)
#     print(down3.shape)
    for temp in templates:
        try:
            res3 = cv2.matchTemplate(down3,temp,cv2.TM_CCORR_NORMED)   
            _, max_val3, _, _ = cv2.minMaxLoc(res3)
            
            res2 = cv2.matchTemplate(down2,temp,cv2.TM_CCORR_NORMED)   
            _, max_val2, _, _ = cv2.minMaxLoc(res2)          
            
            res1 = cv2.matchTemplate(down1,temp,cv2.TM_CCORR_NORMED)   
            _, max_val1, _, _ = cv2.minMaxLoc(res1)  
            
            res = cv2.matchTemplate(src,temp,cv2.TM_CCORR_NORMED)   
            _, max_val, _, _ = cv2.minMaxLoc(res) 
            value.append(max(max_val1,max_val2,max_val3,max_val))
        except:
            value.append(0)
#     print(max(value))
    if max(value) > 0.85:   
        return name[value.index(max(value))]
    else:
        return 'not sure'

    
    

In [3]:
# adapted from lab
def my_skin_detect(src):
    '''
    BGR
    Function that detects whether a pixel belongs to the skin based on RGB values
    Args: 
        src The source color image
    Returns: 
        dst The destination grayscale image where skin pixels are colored white and the rest are colored black
    Surveys of skin color modeling and detection techniques:
    Vezhnevets, Vladimir, Vassili Sazonov, and Alla Andreeva. "A survey on pixel-based skin color detection techniques." Proc. Graphicon. Vol. 3. 2003.
    Kakumanu, Praveen, Sokratis Makrogiannis, and Nikolaos Bourbakis. "A survey of skin-color modeling and detection methods." Pattern recognition 40.3 (2007): 1106-1122.
    '''
    dst = np.zeros(np.shape(src)[:-1], dtype=np.uint8)
    b,g,r = src[:,:,0],src[:,:,1],src[:,:,2]
    mask = np.logical_and.reduce((r>95,b>40,g>20,abs(r-g)>15,r>g,r>b))
    dst[mask]=255
    return dst

def my_frame_differencing(prev, curr):
    '''
    Function that does frame differencing between the current frame and the previous frame
    Args:
        src The current color image
        prev The previous color image
    Returns:
        dst The destination grayscale image where pixels are colored white if the corresponding pixel intensities in the current
    and previous image are not the same
    
    function: cv2.absdiff
    '''
    
    dst = cv2.absdiff(prev, curr)
    gs = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
    dst = (gs > 95).astype(np.uint8) * 255
    return dst


def my_motion_energy(mh):
    '''
    Function that accumulates the frame differences for a certain number of pairs of frames
    Args:
        mh Vector of frame difference images
    Returns:
        dst The destination grayscale image to store the accumulation of the frame difference images
        
        If a pixel has a 255 value in any of the three frames of your motion history, then the outoupt pixel will be set to 255
    ''' 
    dst = np.zeros(np.shape(mh[0]),dtype = np.uint8)
    mask = np.logical_and.reduce((mh[0][:,:] == 255, mh[1][:,:] == 255, mh[2][:,:] == 255,mh[3][:,:] == 255))
    dst[mask] = 255
    return dst

In [5]:
# ----------------
# Reading a stream of images from a webcamera, and displaying the video
# ----------------
# For more information on reading and writing video: http://docs.opencv.org/modules/highgui/doc/reading_and_writing_images_and_video.html
# open the video camera no. 0
cap = cv2.VideoCapture(0)

# if not successful, exit program
if not cap.isOpened():
    print("Cannot open the video cam")
    sys.exit()

# create a window called "MyVideo0"
cv2.namedWindow("Webcam", cv2.WINDOW_AUTOSIZE)

# read a new frame from video
ret, frame0 = cap.read()
if not ret:
    print("Cannot read a frame from video stream")

# show the frame in "Webcam" window
cv2.imshow("Webcam", frame0)

my_motion_history = []
fMH1 = np.zeros(np.shape(frame0)[:-1], dtype=np.uint8)
fMH2 = np.zeros(np.shape(frame0)[:-1], dtype=np.uint8)
fMH3 = np.zeros(np.shape(frame0)[:-1], dtype=np.uint8)
fMH4 = np.zeros(np.shape(frame0)[:-1], dtype=np.uint8)
my_motion_history.append(fMH1)
my_motion_history.append(fMH2)
my_motion_history.append(fMH3)
my_motion_history.append(fMH4)


while(1):
    # read a new frame from video
    ret, frame = cap.read()
    # if not successful, break loop
    if not ret:
        print("Cannot read a frame from video stream")
        break

    cv2.imshow("Webcam", frame)

    # Skin color detection
    frame_dst = my_skin_detect(frame)
    cv2.imshow("Skin", frame_dst)
    
    # Find the largest object and create box for it
    contours, cx, cy = boundingbox(frame_dst)
    cv2.namedWindow("Result", cv2.WINDOW_AUTOSIZE)
    
    # Template matching for detecting different gestures
    ans = my_template_match(frame_dst)
    
    # Display the detection decision on the window
    cv2.putText(contours,ans,(100,300),cv2.FONT_HERSHEY_SIMPLEX,5,(255,255,255),5)
    
    # Background differencing

    # call my_frame_differencing function
    frame_dst = my_frame_differencing(frame0, frame)
    
    # Visualizing motion history
    my_motion_history.pop(0)
    my_motion_history.append(frame_dst)
    
    # call my_motion_energy function
    myMH = my_motion_energy(my_motion_history)
    frame0 = frame
    
    # motion detecting
    if(myMH.any() == True):
        cv2.putText(contours,"moving",(900,300),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2)

    cv2.imshow("Results", contours)
    # wait for 'esc' key press for 30ms. If 'esc' key is pressed, break loop
    if cv2.waitKey(30) == 27:
        print("esc key is pressed by user")
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

esc key is pressed by user


-1